In [1]:
# Some magic so that the notebook will reload the external python script file any time you edit and save the .py file;
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from category_encoders import MEstimateEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import json

In [3]:
# 加载 train.csv 文件
train_file_path = "./train.csv"
train_df = pd.read_csv(train_file_path)

# 确认数据加载成功，并查看数据的前几行
print("数据集前几行：")
print(train_df.head())

数据集前几行：
   listing_id                                              title  \
0     1292132  Land Rover Range Rover Velar 3.0A Si6 R-Dynami...   
1     1294696   Mercedes-Benz C-Class C200 Sport Premium Sunroof   
2     1311717              Honda Odyssey 2.4A (COE till 09/2027)   
3     1310068       Toyota Corolla Altis 1.6A (COE till 12/2028)   
4     1325280                     Lexus GS300 (COE till 06/2026)   

            make    model                                        description  \
0     land rover    range  1 owner, no repairs needed! it looks great, in...   
1  mercedes-benz     c200  rare beautiful white c200 sport premium sunroo...   
2          honda  odyssey            comes with warranty. full service done.   
3         toyota    altis                                                  0   
4          lexus       gs  wear and tear done up. well maintained and reg...   

   manufactured original_reg_date     reg_date  type_of_vehicle  \
0        2018.0               NaN  

In [4]:
# 输出列名以确认数据集结构
print("数据集的列名：", train_df.columns)

数据集的列名： Index(['listing_id', 'title', 'make', 'model', 'description', 'manufactured',
       'original_reg_date', 'reg_date', 'type_of_vehicle', 'category',
       'transmission', 'curb_weight', 'power', 'fuel_type', 'engine_cap',
       'no_of_owners', 'depreciation', 'coe', 'road_tax', 'dereg_value',
       'mileage', 'omv', 'arf', 'opc_scheme', 'lifespan', 'eco_category',
       'features', 'accessories', 'indicative_price', 'price'],
      dtype='object')


In [5]:
# 检查目标列是否存在
required_columns = ['make', 'type_of_vehicle', 'category', 'transmission', 'price']  # 需要处理的列
missing_columns = [col for col in required_columns if col not in train_df.columns]
if missing_columns:
    raise ValueError(f"数据集中缺少以下列：{missing_columns}")


In [6]:
# 3. 使用交叉验证来对 `make` 进行目标编码，防止数据泄漏
def target_encode_cross_validation(df, column, target, n_splits=5, m_value=5.0):
    """对类别特征进行目标编码，并使用交叉验证防止数据泄漏"""
    # 初始化新的编码列，全部填充为 NaN
    df[f"{column}_target_encoded"] = pd.Series([None] * len(df))
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    overall_mean = df[target].mean()  # 计算总体均值（用于平滑）

    for fold_idx, (train_index, val_index) in enumerate(kf.split(df)):
        # 在训练集上拟合目标编码
        train_fold, val_fold = df.iloc[train_index], df.iloc[val_index]
        mean_encoded = train_fold.groupby(column)[target].mean()  # 计算每个类别的均值
        
        # 映射训练集中类别的均值到验证集
        df.loc[val_index, f"{column}_target_encoded"] = val_fold[column].map(mean_encoded)

        # 如果验证集中某些类别未被训练集覆盖，使用总体均值替代
        df[f"{column}_target_encoded"] = df[f"{column}_target_encoded"].infer_objects(copy=False) 

        # 输出每个折的映射结果
        print(f"Fold {fold_idx + 1} 映射结果：")
        print(df.loc[val_index, [column, f"{column}_target_encoded"]].head(10))

    return df

In [7]:
# ============= 1. 对 `make` 使用目标编码 ============= #
train_df = target_encode_cross_validation(train_df, 'make', 'price', n_splits=5, m_value=5.0)

print("\n`make` 目标编码后的结果：")
print(train_df[['make', 'make_target_encoded']].head())

Fold 1 映射结果：
             make  make_target_encoded
17     mitsubishi         65905.048544
29          mazda         63694.364597
30        bentley        475477.576271
34  mercedes-benz        133509.598856
35          honda         71964.719325
44           audi        116195.202368
46     mitsubishi         65905.048544
52            kia         64210.366142
54         toyota         90918.115461
57            NaN                  NaN
Fold 2 映射结果：
             make  make_target_encoded
0      land rover        175544.619318
3          toyota         91211.485772
4           lexus        111378.770149
6     lamborghini        638659.878378
19            NaN                  NaN
31            bmw        123708.002893
38  mercedes-benz        138191.952736
41      ssangyong         56164.558140
42          isuzu         65827.316832
47         toyota         91211.485772
Fold 3 映射结果：
             make  make_target_encoded
5           honda         72665.860512
7         hyundai        

In [8]:
# # ============= 2. 对 `category` 使用频率编码 ============= #
# # 计算每个类别的频率
# category_frequency = train_df['category'].value_counts(normalize=True)
# # 将 `category` 映射为其在数据集中的出现频率
# train_df['category_frequency_encoded'] = train_df['category'].map(category_frequency)
# print("\n`category` 频率编码后的结果：")
# print(train_df[['category', 'category_frequency_encoded']].head())

In [9]:
# 将字符串转换为列表，每个类别作为列表的一个元素
train_df['category_list'] = train_df['category'].apply(lambda x: x.split(', '))

# 初始化MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# 使用MultiLabelBinarizer进行编码
train_df_encoded = mlb.fit_transform(train_df['category_list'])

# 转换回DataFrame并添加列名
train_df_encoded = pd.DataFrame(train_df_encoded, columns=mlb.classes_, index=train_df.index)

# 将编码后的DataFrame合并到原始DataFrame
train_df = pd.concat([train_df, train_df_encoded], axis=1)
train_df = train_df.drop(columns='category')
train_df = train_df.drop(columns='category_list')

print("\n`category` 使用 MultiLabelBinarizer 编码后的结果：")
print(train_df_encoded.head())


`category` 使用 MultiLabelBinarizer 编码后的结果：
   -  almost new car  coe car  consignment car  direct owner sale  \
0  0               0        0                0                  0   
1  0               0        0                0                  0   
2  0               0        1                0                  0   
3  0               0        1                0                  0   
4  0               0        1                0                  0   

   electric cars  hybrid cars  imported used vehicle  low mileage car  \
0              0            0                      0                0   
1              0            0                      0                0   
2              0            0                      0                1   
3              0            0                      0                0   
4              0            0                      0                0   

   opc car  parf car  premium ad car  rare & exotic  sgcarmart warranty cars  \
0        0         1   

In [10]:
# # ============= 3. 对 `type_of_vehicle` 使用标签编码 ============= #
# label_encoder = LabelEncoder()  # 初始化标签编码器
# train_df['type_of_vehicle_label_encoded'] = label_encoder.fit_transform(train_df['type_of_vehicle'])
# print("\n`type_of_vehicle` 标签编码后的结果：")
# print(train_df[['type_of_vehicle', 'type_of_vehicle_label_encoded']].head())

In [11]:
# 初始化OneHotEncoder
onehot_encoder = OneHotEncoder()

# 对type_of_vehicle列进行fit_transform操作，注意要转换为数组
train_df_encoded = onehot_encoder.fit_transform(train_df[['type_of_vehicle']]).toarray()

# 转换回DataFrame，列名使用encoder中的类别名称
train_df_encoded = pd.DataFrame(train_df_encoded, columns=onehot_encoder.get_feature_names_out(['type_of_vehicle']), index=train_df.index)

# 将编码后的DataFrame合并到原始DataFrame
train_df = pd.concat([train_df, train_df_encoded], axis=1)
train_df = train_df.drop(columns='type_of_vehicle')

# 对fuel_type列进行fit_transform操作，注意要转换为数组
train_df_encoded = onehot_encoder.fit_transform(train_df[['fuel_type']]).toarray()

# 转换回DataFrame，列名使用encoder中的类别名称
train_df_encoded = pd.DataFrame(train_df_encoded, columns=onehot_encoder.get_feature_names_out(['fuel_type']), index=train_df.index)

# 将编码后的DataFrame合并到原始DataFrame
train_df = pd.concat([train_df, train_df_encoded], axis=1)
train_df = train_df.drop(columns='fuel_type')

print("\n`type_of_vehicle` 使用 OneHotEncoder 编码后的结果：")
print(train_df.head())


`type_of_vehicle` 使用 OneHotEncoder 编码后的结果：
   listing_id                                              title  \
0     1292132  Land Rover Range Rover Velar 3.0A Si6 R-Dynami...   
1     1294696   Mercedes-Benz C-Class C200 Sport Premium Sunroof   
2     1311717              Honda Odyssey 2.4A (COE till 09/2027)   
3     1310068       Toyota Corolla Altis 1.6A (COE till 12/2028)   
4     1325280                     Lexus GS300 (COE till 06/2026)   

            make    model                                        description  \
0     land rover    range  1 owner, no repairs needed! it looks great, in...   
1  mercedes-benz     c200  rare beautiful white c200 sport premium sunroo...   
2          honda  odyssey            comes with warranty. full service done.   
3         toyota    altis                                                  0   
4          lexus       gs  wear and tear done up. well maintained and reg...   

   manufactured original_reg_date     reg_date transmission  curb_

In [12]:
# ============= 4. 对 `transmission` 进行 One-Hot 编码 ============= #
train_df = pd.get_dummies(train_df, columns=['transmission'], prefix='transmission', drop_first=True)
# train_df.drop(columns='transmission')
print("\n`transmission` One-Hot 编码后的结果：")
print(train_df.head())  # 显示前几行数据


`transmission` One-Hot 编码后的结果：
   listing_id                                              title  \
0     1292132  Land Rover Range Rover Velar 3.0A Si6 R-Dynami...   
1     1294696   Mercedes-Benz C-Class C200 Sport Premium Sunroof   
2     1311717              Honda Odyssey 2.4A (COE till 09/2027)   
3     1310068       Toyota Corolla Altis 1.6A (COE till 12/2028)   
4     1325280                     Lexus GS300 (COE till 06/2026)   

            make    model                                        description  \
0     land rover    range  1 owner, no repairs needed! it looks great, in...   
1  mercedes-benz     c200  rare beautiful white c200 sport premium sunroo...   
2          honda  odyssey            comes with warranty. full service done.   
3         toyota    altis                                                  0   
4          lexus       gs  wear and tear done up. well maintained and reg...   

   manufactured original_reg_date     reg_date  curb_weight  power  ...  \
0  

In [13]:
del_cols = ['listing_id', 'original_reg_date','opc_scheme', 'lifespan','eco_category', 'indicative_price']
text_cols = ['title', 'description', 'features', 'accessories'] # may need further processing
date_cols = ['reg_date'] # need to transform
numeric_cols = ['manufactured', 'curb_weight', 'power', 'engine_cap', 'depreciation', 'coe', 'road_tax', 'dereg_value', 'mileage', 'omv', 'arf', 'price', 'year', 'month']
categorical_cols = ['make', 'model', 'type_of_vehicle', 'category', 'transmission', 'fuel_type', 'no_of_owners']

In [14]:
# Remove unnecessary columns from the dataset
data = train_df.drop(columns=del_cols)
data = data.drop(columns=text_cols)

In [15]:
# 将字符串转换为日期格式
data['reg_date'] = pd.to_datetime(data['reg_date'], format='%d-%b-%Y')

# 提取年份和月份到新的列
data['year'] = data['reg_date'].dt.year
data['month'] = data['reg_date'].dt.month
data = data.drop(columns='reg_date')

In [16]:
for feature in numeric_cols:
    data[feature] = data[feature].fillna(data[feature].median())

long_tail_features = ['omv', 'arf', 'depreciation', 'dereg_value']
for feature in long_tail_features:
    data[f'{feature}'] = np.log1p(data[feature])
    
root_transform_features = ['power', 'engine_cap', 'road_tax', 'mileage']
for feature in root_transform_features:
    data[f'{feature}'] = np.sqrt(data[feature])

In [17]:
max_dict = dict()
min_dict = dict()
for feature in numeric_cols:
    max_dict[feature] = data[feature].max()
    min_dict[feature] = data[feature].min()
    data[feature] = (data[feature] - data[feature].min()) / (data[feature].max() - data[feature].min())

In [18]:
# 5. 保存处理后的数据集
processed_file_path = "./processed_train.csv"
data.to_csv(processed_file_path, index=False)
print(f"处理后的数据已保存至：{processed_file_path}")

with open('max.json', 'w') as f:
    json.dump(max_dict, f)
    
with open('min.json', 'w') as f:
    json.dump(min_dict, f)

处理后的数据已保存至：./processed_train.csv
